# Create overview tables of the data

Create an overview of some metrics for all tables in RWD.

In [ ]:
import os
import sys

import pandas as pd

# for Jupyter notebooks: add the path of 'code' to allow importing module
sys.path.append(os.path.join(os.getcwd(), ".."))
from afd_measures import utils as afd_utils

data_path = "../../data"
gt_path = "../../data/ground_truth.csv"
results_path = "../../results"

rwd_data = {}
for file in filter(lambda f: f.endswith(".csv"), os.listdir(os.path.join(data_path, "rwd"))):
    rwd_data[file] = pd.read_csv(os.path.join(data_path, "rwd", file))
    rwd_data[file].columns = [
        afd_utils.clean_colname(c) for c in rwd_data[file].columns
    ]

rwd_results = pd.DataFrame()
for file in filter(
    lambda f: f.startswith("rwd_results_") and f.endswith(".csv"),
    os.listdir(results_path),
):
    rwd_results = pd.concat(
        [rwd_results, pd.read_csv(os.path.join(results_path, file))]
    )

## Create an overview table

In [ ]:
name_mapping = {
    "adult.csv": "\\adultData",
    "claims.csv": "\\claimsData",
    "dblp10k.csv": "\\dblpData",
    "hospital.csv": "\\hospitalData",
    "tax.csv": "\\taxData",
    "t_biocase_gathering_agent_r72738_c18.csv": "\\gathAgentData",
    "t_biocase_gathering_namedareas_r137711_c11.csv": "\\gathAreaData",
    "t_biocase_gathering_r90992_c35.csv": "\\gathData",
    "t_biocase_identification_highertaxon_r562959_c3.csv": "\\identTaxonData",
    "t_biocase_identification_r91800_c38.csv": "\\identData",
}
data = {}
for table in rwd_results.loc[:, "table"].unique():
    df = rwd_data[table]
    _results = rwd_results.query("table == @table")
    data[name_mapping[table]] = {
        "rows": df.shape[0],
        "attributes": df.columns.size,
        "candFDs": _results.query("g3_prime >= .5").shape[0],
        "FDs": _results.query("(exact_fd == True) & (afd == True)").shape[0],
        "AFDs": _results.query("(exact_fd == False) & (afd == True)").shape[0],
    }
pd.DataFrame(data).T.to_csv(
    "../../paper/table2_dataset_descriptions.csv",
    index_label="Relation R",
)

## Calculate counts

In [ ]:
{
    "number of tables in RWD": rwd_results["table"].unique().size,
    "total number of column combinations": rwd_results.shape[0],
    "number of canidate FDs in RWD": rwd_results.query("g3_prime >= .5").shape[0],
    "ground truth size": rwd_results.query("afd == True").shape[0],
    "number of exact FDs in ground truth": rwd_results.query(
        "(exact_fd == True) and (afd == True)"
    ).shape[0],
    "number of AFDs in ground truth": rwd_results.query(
        "(exact_fd == False) and (afd == True)"
    ).shape[0],
    "number of measures": len(afd_utils.measure_order),
    "number of non exact FDs": rwd_results.query("(exact_fd == False)").shape[0],
}